# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [16]:
# Import necessary libraries
from google.colab import files, drive
import zipfile
import shutil
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Activation, Flatten, MaxPooling2D, Conv2D
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from tensorflow.keras.applications import MobileNetV2
from collections import Counter
import random
from matplotlib.backends.backend_pdf import PdfPages
import json
import cv2
from PIL import Image
import pandas as pd

In [3]:
drive.mount('/content/drive')

# Path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/ML_Tensorflow/phytoplankton_labeled.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

data_dir = '/content/dataset/labeled_20201020'

base_dir = '/content/dataset/groups'

Mounted at /content/drive


In [9]:
# Import models

model_dir = '/content/drive/MyDrive/ML_Tensorflow/models/'

child_models = [load_model(model_dir + 'model_0_9.h5'), load_model(model_dir + 'model_10_19.h5'), load_model(model_dir + 'model_20_29.h5'), load_model(model_dir + 'model_30_39.h5'), load_model(model_dir + 'model_40_49.h5')]

parent_model = load_model(model_dir + 'model_parent.h5')

print("Finished loading the models.")

Finished loading the models.


## Model Selection

We decided to employ a cascading model architecture with Transfer learning based off of the MobileNetV2 architecture trained on 'imagenet'. The models are all Convolutional Neural Networks that classify the phytoplankton image into classes. Initially the parent model would classify into one of 5 groups, which represents one of 5 child models, that would then be invoked to classify into its one of ten classes.


## Feature Engineering

The exact steps are demonstrated in the respective JupyterNotebooks for each model. However as CNNs extract the features using the model architecture, we are using MobileNetV2's feature extraction capabilities and apply them to our dataset.


In [27]:
# Initialize lists to hold image paths and labels
image_paths = []
labels = []

# Iterate over each sub-directory in the dataset directory
for label_dir in os.listdir(data_dir):
    label_dir_full_path = os.path.join(data_dir, label_dir)

    # Check if it's a directory
    if os.path.isdir(label_dir_full_path):
        # Iterate over each image in the sub-directory
        for image_file in os.listdir(label_dir_full_path):
            # Construct the full path for the image file
            image_file_full_path = os.path.join(label_dir_full_path, image_file)

            # Check if it's a file and not a directory
            if os.path.isfile(image_file_full_path):
                # Add the image path and label to the lists
                image_paths.append(image_file_full_path)
                labels.append(label_dir) # assuming label is the directory name

# Now you can split the image paths and labels into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(image_paths, labels, test_size=0.3, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

unique_classes = np.unique(labels)

# Generate a dictionary mapping numerical values to class names
class_to_idx = {class_name: idx for idx, class_name in enumerate(unique_classes)}

# Generate a dictionary mapping numerical values back to class names
idx_to_class = {idx: class_name for class_name, idx in class_to_idx.items()}

print(f"Number of images in dataset: {len(image_paths)}")

Number of images in dataset: 63074


In [28]:
batch_size = 32

# Convert the lists into DataFrames
train_df = pd.DataFrame({'filepath': X_train, 'label': y_train})
val_df = pd.DataFrame({'filepath': X_val, 'label': y_val})
test_df = pd.DataFrame({'filepath': X_test, 'label': y_test})

# Initialize ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=(160, 160),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 9462 validated image filenames belonging to 50 classes.


In [29]:
# Function to predict class for a batch of images
def predict_classes(images, parent_model, child_models):
    # Predict the group using the parent model
    group_probabilities = parent_model.predict(images)
    groups = np.argmax(group_probabilities, axis=1)
    print(f"Image was predicted to belong to group {groups}.")

    final_predictions = []

    # Iterate over each image and its predicted group
    for image, group in zip(images, groups):
        # Prepare image for prediction (reshape to include batch dimension)
        image_batch = np.expand_dims(image, axis=0)

        # Predict the final class using the corresponding child model
        final_class_probabilities = child_models[group].predict(image_batch)
        final_class = np.argmax(final_class_probabilities, axis=1)[0]
        final_predictions.append(final_class)

    return final_predictions

# Predicting on test set
predictions = []
for images, _ in test_generator:
    batch_predictions = predict_classes(images, parent_model, child_models)
    predictions.extend(batch_predictions)

    # Break the loop if we have predicted for all images
    if len(predictions) >= len(test_df) or len(predictions) >= 5:
        break

# Truncate predictions to match the length of test set (in case of last partial batch)
predictions = predictions[:len(test_df)]

# Convert labels back to original format if necessary
# e.g., labels = [class_mapping[p] for p in predictions]

# Calculate accuracy
correct_predictions = sum(p == actual for p, actual in zip(predictions, test_df['label']))
accuracy = correct_predictions / len(test_df)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")


1/1 [==============================] - 1s 761ms/step
Image was predicted to belong to group [3 4 1 1 1 2 0 4 1 1 1 3 3 2 2 1 3 1 0 1 0 2 2 4 1 4 4 0 3 1 4 4].
1/1 [==============================] - 0s 46ms/step
Accuracy on test set: 0.00%


In [34]:
def get_class_name_from_index(index, idx_to_class):
    """
    Get the class name corresponding to a numerical index.

    :param index: The numerical index (prediction from the model).
    :param idx_to_class: A dictionary mapping indices to class names.
    :return: The corresponding class name.
    """
    return idx_to_class.get(index, "Unknown")

def load_and_preprocess_image(image_path, target_size=(160, 160)):
    """
    Load an image and preprocess it for model prediction.

    :param image_path: Path to the image file.
    :param target_size: The target size to which the image is resized. Default is (160, 160).
    :return: Preprocessed image.
    """
    # Open the image file
    with Image.open(image_path) as img:
        # Resize the image
        img = img.resize(target_size)

        # Convert the image to a numpy array
        image_array = np.array(img)

        # Ensure the image has 3 channels (RGB)
        if image_array.ndim == 2:
            image_array = np.stack((image_array,)*3, axis=-1)
        elif image_array.shape[2] == 4:
            image_array = image_array[..., :3]

        # Normalize the image
        image_array = image_array / 255.0

    return image_array

# Function to predict the class for a single image
def predict_class(image, parent_model, child_models):
    # Prepare image for prediction (adding batch dimension)
    image_batch = np.expand_dims(image, axis=0)

    # Predict the group using the parent model
    group_probabilities = parent_model.predict(image_batch)
    group = np.argmax(group_probabilities, axis=1)[0]
    print(f"Image classified as belonging to group {group}")

    # Select the child model based on the predicted group
    child_model = child_models[group]

    # Predict the final class using the child model
    final_class_probabilities = child_model.predict(image_batch)
    final_class = np.argmax(final_class_probabilities, axis=1)[0]

    return final_class

# Predict for 5 images from the test set
num_samples_to_predict = 20
sample_predictions = []

for i in range(num_samples_to_predict):
    image_path = test_df.iloc[i]['filepath']  # Get the image path
    true_label = test_df.iloc[i]['label']  # Get the true label for comparison
    image = load_and_preprocess_image(image_path)  # Load and preprocess the image

    prediction = predict_class(image, parent_model, child_models)  # Predict the class
    sample_predictions.append((image_path, prediction, true_label))  # Store the results

# Display the predictions
for img_path, pred, true in sample_predictions:
    class_name = get_class_name_from_index(pred, idx_to_class)
    print(f"Predicted class: {class_name}, True class: {true}")


1/1 [==============================] - 0s 43ms/step
Image classified as belonging to group 3
1/1 [==============================] - 0s 49ms/step
Image classified as belonging to group 4
1/1 [==============================] - 0s 44ms/step
Image classified as belonging to group 1
1/1 [==============================] - 0s 43ms/step
Image classified as belonging to group 1
1/1 [==============================] - 0s 46ms/step
Image classified as belonging to group 1
1/1 [==============================] - 0s 74ms/step
Image classified as belonging to group 1
1/1 [==============================] - 0s 67ms/step
Image classified as belonging to group 0
1/1 [==============================] - 0s 69ms/step
Image classified as belonging to group 1
1/1 [==============================] - 0s 66ms/step
Image classified as belonging to group 1
1/1 [==============================] - 0s 74ms/step
Image classified as belonging to group 1
1/1 [==============================] - 0s 55ms/step
Image classified a

## Hyperparameter Tuning

We found that for our hardware, training for about 15 epochs was feasible as well as achieving great results. We also tested out changes to the learning rate, batch size and target number of images per class. Having too many images per class lead to overfitting for those that we only had less than 30 "real" images of. Slowly iterating through a combination of those hyperparameters had us reach 175 images per class.
Of course the majority of the hyperparameters were given by MobileNetV2.


## Implementation

The model implementations can be found [here](./JupyterNotebooks/).

## Evaluation Metrics

We initially focussed on test accuracy for the 5 child models. After evaluating the results, we paid more attention to recall and precision (F1-Score being their combination). Those metrics were super helpful for deciding whether we have potential for optimizing the model further. We visualised the results with a confusion matrix. Those can also be found in the respective JuypterNotebooks:
* [Range_0_9](./JupyterNotebooks/MobileNetV2_0_9.ipynb)
* [Range_10_19](./JupyterNotebooks/MobileNetV2_10_19.ipynb)
* [Range_20_29](./JupyterNotebooks/MobileNetV2_20_29.ipynb)
* [Range_30_39](./JupyterNotebooks/MobileNetV2_30_39.ipynb)
* [Range_40_49](./JupyterNotebooks/MobileNetV2_40_49.ipynb)

## Comparative Analysis

The overall parent model is performing very poorly, unable to classify into one of 5 groups. However, the models for classification into one of 10 classes outperform the baseline model by far, reaching F1-Scores of +95% in some instances.
